**SVM**

**Exercise 1**

In [23]:

from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

In [24]:
iris = load_iris()
data_set = iris.data
labels = iris.target

def build_kernel(data_set, d, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = kernel**d
    return kernel

In [25]:
build_kernel(data_set,2 , kernel_type='poly')

array([[1620.8676, 1405.5001, 1371.2209, ..., 2634.7689, 2656.3716,
        2312.6481],
       [1405.5001, 1225.7001, 1189.5601, ..., 2355.1609, 2361.96  ,
        2062.0681],
       [1371.2209, 1189.5601, 1160.0836, ..., 2238.2361, 2256.25  ,
        1964.2624],
       ...,
       [2634.7689, 2355.1609, 2238.2361, ..., 6771.6441, 6918.9124,
        6001.6009],
       [2656.3716, 2361.96  , 2256.25  , ..., 6918.9124, 7131.8025,
        6155.9716],
       [2312.6481, 2062.0681, 1964.2624, ..., 6001.6009, 6155.9716,
        5337.7636]])

**Excercise 2**

In [26]:
def choose_set_for_label(data_set, label):
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, label)
    return train_data_set, test_data_set, train_labels, test_labels

In [27]:
def get_labels_count(data_set):
    return len(data_set)

In [28]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [29]:
# modify this part 

data_set = data_set[labels != 2]
labels = labels[labels != 2]
train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels)
objects_count = get_labels_count(train_labels)
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0: -3.3696e+03 -2.1055e+04  2e+05  4e+00  8e-15
 1: -9.1498e+02 -1.5605e+04  3e+04  7e-01  5e-14
 2: -2.7036e+02 -4.0718e+03  5e+03  5e-02  4e-14
 3: -2.4353e+02 -4.8563e+02  3e+02  3e-03  2e-14
 4: -3.6233e+02 -3.9156e+02  3e+01  2e-04  2e-14
 5: -3.6992e+02 -3.7022e+02  3e-01  2e-06  1e-14
 6: -3.7000e+02 -3.7000e+02  3e-03  2e-08  1e-14
 7: -3.7000e+02 -3.7000e+02  3e-05  2e-10  1e-14
Optimal solution found.
0.52
